In [6]:
import astropy.units as u
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ReadFile import Read
from CenterOfMass import CenterOfMass

In [7]:
# NOTE: I am repeatedly running into a particular error: Can only apply 'greater' function to dimensionless 
# quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
class MassProfile:
    
    def __init__(self, galaxy, Snap):
        
        ilbl = '000' + str(Snap)
        ilbl = ilbl[-3:]
        self.filename = "%s_" %(galaxy) + ilbl + '.txt'
        
        self.time, self.total, self.data = Read(self.filename)
        self.m = self.data['m']
        self.x = self.data['x'] * u.kpc
        self.y = self.data['y'] * u.kpc
        self.z = self.data['z'] * u.kpc
        
        self.gname = galaxy
    
    def COMdefine(self,a,b,c,m):
        #INPUTS: a,b,c: can be positions or velocities.
        # m = mass
        # RETURNS: the components of the center of mass position or velocity.
        Acom = np.sum(a*m)/np.sum(m)
        Bcom = np.sum(b*m)/np.sum(m)
        Ccom = np.sum(c*m)/np.sum(m)
        return Acom, Bcom, Ccom
        
        # function which returns the mass enclosed in a particular radius.
    def MassEnclosed(self, ptype, Radius):
        # INPUTS: ptpye = particle type.
        # Radius = It is an array with radii in kpc.
        # RETURNS: an array of enclosed mass within a radius (in kpc)
            
        # defining an object (COM) and calling COM_P from CenterOfMass class.
        COM = CenterOfMass.COM_P(self, 0.1) 
        #print(COM)
            
        # initializing an array to be used in the loop.
        MassWithinR = np.zeros(1)
        
        # a for loop over the radii's elements to print out the masses within a particular radii.
        for i in Radius:
            R = np.linalg.norm(COM)
            index = np.where(i <= R)
            MassWithinR = sum(self.m[index])
                
        return MassWithinR 
        
    def MassEnclosedTotal(self, radius_tot):
        #INPUTS: an array of radii (in kpc) 
        # RETURNS: an array of total enclosed mass (in Msun)
        
        
        # initializing the array to be used in a for loop.
        TotalMass = np.zeros(1)
         
        # a for loop over the total radii.
        for i in radius_tot:
            
            index2 = sum(self.MassEnclosed(ptype, i))
            TotalMass = self.m[index2]
                
        return TotalMass
     
    # calculating the mass enclosed within a theoretical radius (in kpc) using the Hernquist mass profile.
    def HernquistMass(self, r, a, Mhalo):
        return Mhalo*r**2/(a+r)**2 *u.Msun
    
    # module to calculate an array of circular velocities (in km/s) of a ptype within a given radius (in kpc).
    def CircularVelocity(self, ptype, radius):
        #INPUTS: ptype = particle type.
        #radius = an array of radii (in kpc)
        G = 6.67e-11 # the gravitational constant
        circ_vel = np.zeros(1) # initializing an array with zero.
       
        # a for loop over different radii in the radius array to calculate the circular velocities (in km/s).    
        for i in radius:
            circ_vel = np.sqrt(G*self.MassEnclosed(ptype,i)/i)
                
        return circ_vel *u.km/u.s
    # module to calculate an array of total circular velocities (in km/s) of all ptypes in a given radius.    
    def TotalCircularVelocity(self, radius):
        # INPUTS: radius = an array of radii (in kpc).
        # RETURNS: an array of total circular velocity (in km/s) within a given radius.
        
        G = 6.67e-11 # the gravitational constant
        TotalV_circ = np.zeros(1)
            
        
        # for loop over the radii elements to calculate the total circular velocities (in km/s).   
        for i in radius:
            TotalV_circ = np.sqrt(G*self.TotalMassEnclosed(i)/i)
                
        return TotalV_circ
     
    # module to calculate the circular speed using the Hernquist mass profile
    def HernquistVCirc(self, radius, a, Mhalo):
        G = 6.67e-11
            
        return np.sqrt(G*self.HernquistMass (radius, a, Mhalo)/radius)

In [8]:
# testing my code above with an arbitrary array of radii.
MW = MassProfile("MW",0)
Radius = np.arange(0.25,30.5,1.5)
print(Radius)
MW.MassEnclosed(1, Radius)

[ 0.25  1.75  3.25  4.75  6.25  7.75  9.25 10.75 12.25 13.75 15.25 16.75
 18.25 19.75 21.25 22.75 24.25 25.75 27.25 28.75 30.25]


UnitConversionError: Can only apply 'greater' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)

In [ ]:
#initializing the plotting canvas.
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
# the values of radii to be plotted.
r = np.arange(0.1,30,1)
# setting the y-axis in log units.
ax.semilogy(r, MassProfile.MassEnclosedTotal(r), color = 'red', linewidth = 5, label = 'MW')
# labeling the x and y axes.
plt.xlabel('Radius', fontsize=22)
plt.ylabel('Mass Enclosed', fontsize=22)
# range of values of the x-axis.
plt.xlim(0,35)
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

# added a legend.
legend = ax.legend(loc='upper right',fontsize='x-large')

